# Features

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import graphlab as gl
import sklearn as sk
import time
from util import *
start_time = time.time()

In [24]:
# def convert_data_to_utf8(files):
#     files = ['./data/'+file for file in files]
#     for file in files:
#         df = pd.read_csv(file, encoding='ISO-8859-1')
#         df.to_csv(file[:-4]+'_utf8.csv', encoding='utf-8')
        
# convert_data_to_utf8(['train.csv', 'attributes.csv', 'product_descriptions.csv', 'test.csv'])        

## Load Files

In [6]:
df_train = gl.SFrame.read_csv('./data/train_utf8.csv')
df_test = gl.SFrame.read_csv('./data/test_utf8.csv')
df_attr = gl.SFrame.read_csv('./data/attributes_utf8.csv', column_type_hints=[long,long,str,str])
df_desp = gl.SFrame.read_csv('./data/product_descriptions_utf8.csv')

2016-04-08 21:12:46,153 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: C:\Users\linghao\AppData\Local\Temp\graphlab_server_1460121164.log.0


Finished parsing file C:\Home\Projects\Kaggle\Playground\home-depot\data\train_utf8.csv

Parsing completed. Parsed 100 lines in 0.201011 secs.

This non-commercial license of GraphLab Create is assigned to zhanglh13@fudan.edu.cn and will expire on September 21, 2016. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file C:\Home\Projects\Kaggle\Playground\home-depot\data\train_utf8.csv

...

Parsing completed. Parsed 100 lines in 0.504029 secs.


Inferred types from first line of file as 
column_type_hints=[long,long,long,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file C:\Home\Projects\Kaggle\Playground\home-depot\data\product_descriptions_utf8.csv

Parsing completed. Parsed 124428 lines in 1.15807 secs.


Inferred types from first line of file as 
column_type_hints=[long,long,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [7]:
num_train = len(df_train)
df_test['relevance'] = 0.0
df = df_train.append(df_test)
df_brand = df_attr[df_attr['name'] == 'MFG Brand Name'][['product_uid', 'value']].rename({'value': 'brand'})
df = df.join(df_desp, on='product_uid', how='left')
df = df.join(df_brand, on='product_uid', how='left')
# print('--- Files Loaded: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

## Fix Typo

In [8]:
df['search_term'] = df['search_term'].apply(correct_typo)
# print("--- Typo Fixed: %s minutes ---" % round(((time.time() - start_time) / 60), 2))

## Stemming

In [9]:
df['search_term'] = df['search_term'].apply(lambda x: stem_str(x))
df['product_title'] = df['product_title'].apply(lambda x: stem_str(x))
df['product_description'] = df['product_description'].apply(lambda x: stem_str(x))
df['brand'] = df['brand'].apply(lambda x: stem_str(x))
# print('--- Stemming: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

## Meat-Features

### Length of Texts

In [10]:
df['len_search_term'] = df['search_term'].apply(lambda x: len(x.split()))
df['len_product_title'] = df['product_title'].apply(lambda x: len(x.split()))
df['len_product_description'] = df['product_description'].apply(lambda x: len(x.split()))
df['len_brand'] = df['brand'].apply(lambda x: len(x.split()))
# print('--- Length of Texts: %s minutes ---' % round(((time.time() - start_time) / 60), 2))

## TF-IDF

In [11]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df = 0, stop_words = 'english', encoding='utf-8', decode_error='ignore')
# tfidf_mat_product_title = tfidf.fit_transform(df['product_title'])

In [12]:
df['tfidf_product_title'] = gl.text_analytics.tf_idf(df['product_title'])

In [13]:
df['top20_tfidf_product_title'] = df['tfidf_product_title'].apply(lambda x: zip(*sorted([(v, k) for (k, v) in x.iteritems()], reverse=True)[:20])[1])

In [95]:
columns_to_be_tfidf_vectorized = ['product_title', 'product_description', 'brand']

In [ ]:
for col in columns_to_be_tfidf_vectorized:
    pass

In [ ]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
print("--- Prod Info: %s minutes ---" % round(((time.time() - start_time)/60),2))

In [ ]:
df_all['search_term'] = df_all['product_info'].map(lambda x:seg_words(x.split('\t')[0],x.split('\t')[1]))
print("--- Search Term Segment: %s minutes ---" % round(((time.time() - start_time)/60),2))

df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
print("--- Query In: %s minutes ---" % round(((time.time() - start_time)/60),2))

df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))

## Text Similarities

In [14]:
def sum_value(array):
    return sum(filter(None, array))

def max_value(array):
    try:
        return max(filter(None, array))
    except:
        return 0

def min_value(array):
    try:
        return min(filter(None, array))
    except:
        return 0

### Levenshtein Distance

Lev-Dist Vector -> min/sum  
Keep L-D Vector or not?

In [15]:
import Levenshtein as lv
def lev_dist(text1, text2):
    return [lv.distance(x, y) for x in text1 for y in text2]

In [16]:
df['lev_dist_to_product_title_list'] = df.apply(lambda x: lev_dist(x['search_term'].split(), x['top20_tfidf_product_title']))

In [17]:
df['lev_dist_to_product_title_min'] = df['lev_dist_to_product_title_list'].apply(lambda x: min_value(x))
df['lev_dist_to_product_title_max'] = df['lev_dist_to_product_title_list'].apply(lambda x: max_value(x))
df['lev_dist_to_product_title_sum'] = df['lev_dist_to_product_title_list'].apply(lambda x: sum_value(x))

### Word2Vec

## Filling NAs

In [48]:
def find_columns_with_na(df):
    ret = []
    for col in df.column_names():
        print 'checking ', col
        num_na = df[col].sketch_summary().num_undefined()
        if num_na > 0:
            ret.append(col)
            print col, 'has missing values!'

na_cols = find_columns_with_na(df)

checking  X1
checking  id
checking  product_uid
checking  product_title
checking  search_term
checking  relevance
checking  product_description
checking  brand
brand
checking  len_search_term
checking  len_product_title
checking  len_product_description
checking  len_brand
checking  lev_dist_to_product_title_min
checking  lev_dist_to_product_title_max
checking  lev_dist_to_product_title_sum


In [49]:
df = df.fillna('brand', 0)

## Export

In [50]:
columns_to_remove = ['X1.1', 'tfidf_product_title', 'top20_tfidf_product_title', 'lev_dist_to_product_title_list']

In [51]:
for col in columns_to_remove:
    try:
        df.remove_column(col)
    except:
        pass

In [54]:
df.save('./df_.csv', format='csv')